In [1]:
# This needs to be here because by default Jupyter only adds the pwd to sys.path
import os, sys
if os.path.abspath('..') not in sys.path: sys.path.append(os.path.abspath('..'))

import torch
import time
import scipy
import pickle 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from pysrc.constants import datapath, N_ITEMS, N_USERS, N_ITEMS_SUB, N_USERS_SUB
from torch.utils.data import Dataset
from scipy.sparse import csr_matrix, coo_matrix
from pathlib import Path
from collections import defaultdict

c:\users\nick\appdata\local\programs\python\python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for dir_name in ["uniform", "top", "middle", "LGR", "UIS1"]:
    Path(datapath("train/" + dir_name)).mkdir(parents=True, exist_ok=True)
    Path(datapath("test/" + dir_name)).mkdir(parents=True, exist_ok=True)

### Full Dataset (No Sparsity)

In [3]:
user_ids = []
item_ids = []
values = []
with open(datapath("train.txt")) as file:
    i = 0
    for line in file:
        user_id, items = line.split(maxsplit=1)
        items = [int(id) for id in items.split()]
        user_ids += [int(user_id)] * len(items)
        values += [1] * len(items)
        item_ids += items
        i += 1
        if i > 1000:
            break

train_data = torch.sparse_coo_tensor([user_ids, item_ids], values, (1001, N_ITEMS))
torch.save(train_data, datapath("full_data.pt"))
print(train_data)

tensor(indices=tensor([[    0,     0,     0,  ...,  1000,  1000,  1000],
                       [    0,     1,     2,  ...,  7856,  8922, 24260]]),
       values=tensor([1, 1, 1,  ..., 1, 1, 1]),
       size=(1001, 91599), nnz=109986, layout=torch.sparse_coo)


In [4]:
test_dict = {}
with open(datapath("test.txt")) as file:
    i = 0
    for line in file:
        split = line.split(maxsplit=1)
        if (len(split) > 1):
            (user_id, items) = split
            test_dict[user_id] = items.split()
        else:
            test_dict[split[0]] = ''
        i += 1
        if i > 1000:
            break
test_dict = {int(k):[int(id) for id in v] for k,v in test_dict.items()}
with open(datapath("full_data_test_indices20.pickle"), "wb") as f:
    pickle.dump(test_dict, f)

In [5]:
np.random.seed(seed = 0)

for sparsity in [0]:
    user_ids = []
    item_ids = []
    values = []
    with open(datapath("train.txt")) as file:
        for line in file:
            user_id, items = line.split(maxsplit=1)
            items = [int(id) for id in items.split()]
            user_ids += [int(user_id)] * len(items)
            values += [1] * len(items)
            item_ids += items

    indices = np.random.choice(range(len(user_ids)), 
                               size = int(len(user_ids) * (1 - sparsity)), 
                               replace = False)
    user_ids = np.array(user_ids)[indices]
    item_ids = np.array(item_ids)[indices]
    values = np.array(values)[indices]

    train_data = torch.sparse_coo_tensor(np.array([user_ids, item_ids]), values, (N_USERS, N_ITEMS))
    torch.save(train_data, datapath(f"train/uniform/uniform{int(100*(1 - sparsity))}_data.pt"))

    test_dict = {}
    with open(datapath("test.txt")) as file:
        for line in file:
            split = line.split(maxsplit=1)
            if (len(split) > 1):
                (user_id, items) = split
                test_dict[user_id] = items.split()
            else:
                test_dict[split[0]] = ''

    test_dict = {int(k):[int(id) for id in v] for k,v in test_dict.items()}
    with open(datapath(f"test/uniform/uniform{int(100*(1 - sparsity))}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)

### Uniform Degradation

In [6]:
np.random.seed(seed = 0)

for sparsity in [0, .01, .02, .03, .05, .1, .2, .3, .4, .5, .6, .7, .9]:
    user_ids = []
    item_ids = []
    values = []
    with open(datapath("train.txt")) as file:
        for line in file:
            user_id, items = line.split(maxsplit=1)
            items = [int(id) for id in items.split()]
            user_ids += [int(user_id)] * len(items)
            values += [1] * len(items)
            item_ids += items

    indices = np.random.choice(range(len(user_ids)), 
                               size = int(len(user_ids) * (1 - sparsity)), 
                               replace = False)
    user_ids = np.array(user_ids)[indices]
    item_ids = np.array(item_ids)[indices]
    values = np.array(values)[indices]

    train_data = torch.sparse_coo_tensor(np.array([user_ids, item_ids]), values, (N_USERS, N_ITEMS))
    torch.save(train_data, datapath(f"train/uniform/uniform{int(100*(1 - sparsity))}_data.pt"))

    test_dict = {}
    with open(datapath("test.txt")) as file:
        for line in file:
            split = line.split(maxsplit=1)
            if (len(split) > 1):
                (user_id, items) = split
                test_dict[user_id] = items.split()
            else:
                test_dict[split[0]] = ''

    test_dict = {int(k):[int(id) for id in v] for k,v in test_dict.items()}
    with open(datapath(f"test/uniform/uniform{int(100*(1 - sparsity))}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\nick\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_21184\3122899315.py", line 18, in <module>
    user_ids = np.array(user_ids)[indices]
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nick\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\users\nick\appdata\local\programs\python\python37\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_

TypeError: object of type 'NoneType' has no len()

### Generic Setup

In [ ]:
user_ids = []
item_ids = []
values = []
with open(datapath("test.txt")) as file:
    for line in file:
        split = line.split(maxsplit=1)
        if (len(split) > 1):
            (user_id, items) = split
            items = [int(id) for id in items.split()]
            user_ids += [int(user_id)] * len(items)
            values += [1] * len(items)
            item_ids += items

train_data = torch.load(datapath("full_data.pt"))
test_data = torch.sparse_coo_tensor([user_ids, item_ids], values, (N_USERS, N_ITEMS))
all_data = train_data + test_data
all_data

In [ ]:
def select_columns_coo(coo, cols):
    coo_indices = coo.coalesce().indices().numpy()
    selected_indices = coo_indices[:, np.isin(coo_indices[1, :], cols)]

    coo_values = coo.coalesce().values().numpy()
    selected_values = coo_values[np.isin(coo_indices[1, :], cols)]

    cols = np.array(cols)
    adj = {}
    for i in range(coo.shape[1]):
        adj[i] = i - np.sum(cols < i)

    for i in range(selected_indices.shape[1]):
        selected_indices[1, i] -= adj[selected_indices[1, i]]

    return torch.sparse_coo_tensor(selected_indices, selected_values, (coo.shape[0], len(cols)))

In [ ]:
# Simple unit test
i = [[0, 1, 1, 2], [0, 1, 3, 0]]
v = [1, 2, 1, 3]

T = torch.sparse_coo_tensor(i, v, (3, 5))
print(T.to_dense())
print(select_columns_coo(T, [0, 2, 3, 4]).to_dense())

In [ ]:
def sparse_to_user_dict(coo):
    coo_indices = coo.coalesce().indices().numpy()
    out_dict = {}
    for i in range(N_USERS):
        out_dict[i] = list(coo_indices[1, coo_indices[0, :] == i])
    return out_dict

In [ ]:
# Simple unit test
i = [[0, 1, 1, 2], [0, 1, 3, 0]]
v = [1, 2, 1, 3]

T = torch.sparse_coo_tensor(i, v, (3, 5))
print(T.to_dense())
print(sparse_to_user_dict(T))

### Top (Item) Pruning

In [ ]:
inds = all_data.coalesce().indices().numpy()
X = csr_matrix((np.ones(inds.shape[1]), (inds[0, :], inds[1, :])), shape = (N_USERS, N_ITEMS))
totals = X.sum(axis = 0).A.flatten()

item_sparsities = {}
for i in range(len(totals)):
    item_sparsities[i] = totals[i] / N_ITEMS
qs = np.array([list(item_sparsities.keys()), list(item_sparsities.values())])

for q in tqdm([70, 75, 80, 85, 90, 95]):
    quant = np.quantile(qs[1, :], q * 0.01)
    indices = qs[0, qs[1, :] >= quant]
    
    dense_train_data = train_data.bool().to_dense().clone()
    dense_test_data = test_data.bool().to_dense().clone()
    dense_train_data[:, indices] = 0
    dense_test_data[:, indices] = 0
    torch.save(dense_train_data.to_sparse(), datapath(f"train/top/top{q}_data.pt"))
    
    test_dict = sparse_to_user_dict(dense_test_data.to_sparse())

    with open(datapath(f"test/top/top{q}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)


### Middle Pruning

In [ ]:
inds = all_data.coalesce().indices().numpy()
X = csr_matrix((np.ones(inds.shape[1]), (inds[0, :], inds[1, :])), shape = (N_USERS, N_ITEMS))
totals = X.sum(axis = 0).A.flatten()

item_sparsities = {}
for i in range(len(totals)):
    item_sparsities[i] = totals[i] / N_ITEMS
qs = np.array([list(item_sparsities.keys()), list(item_sparsities.values())])

for radius in tqdm([5, 10, 15, 20, 25, 30]):
    low = np.quantile(qs[1, :], 0.5 - radius * 0.01)
    high = np.quantile(qs[1, :], 0.5 + radius * 0.01)
    indices = qs[0, (qs[1, :] < low) | (qs[1, :] > high)]

    dense_train_data = train_data.bool().to_dense().clone()
    dense_test_data = test_data.bool().to_dense().clone()
    dense_train_data[:, indices] = 0
    dense_test_data[:, indices] = 0
    torch.save(dense_train_data.to_sparse(), datapath(f"train/middle/middle{radius}_data.pt"))
    
    test_dict = sparse_to_user_dict(dense_test_data.to_sparse())

    with open(datapath(f"test/middle/middle{radius}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)

### LGR and UIS1 Degredation

Reference: https://www.sciencedirect.com/science/article/pii/S0957417410010985

In [ ]:
# Calculate Jaccard matrix between users and determine connected components of user graph

def pairwise_jaccard_sparse(csr, epsilon):
    """
    Reference: https://stackoverflow.com/questions/32805916/compute-jaccard-distances-on-sparse-matrix
    Computes the Jaccard distance between the rows of `csr`, smaller than the cut-off distance `epsilon`.
    """
    assert(0 < epsilon < 1)
    csr = csr_matrix(csr).astype(bool).astype(int)

    csr_rownnz = csr.getnnz(axis=1)
    intrsct = csr.dot(csr.T)

    nnz_i = np.repeat(csr_rownnz, intrsct.getnnz(axis=1))
    unions = nnz_i + csr_rownnz[intrsct.indices] - intrsct.data
    dists = 1.0 - intrsct.data / unions

    mask = (dists > 0) & (dists <= epsilon)
    data = dists[mask]
    indices = intrsct.indices[mask]

    rownnz = np.add.reduceat(mask, intrsct.indptr[:-1])
    indptr = np.r_[0, np.cumsum(rownnz)]

    out = csr_matrix((data, indices, indptr), intrsct.shape)
    return out

inds = all_data.coalesce().indices().numpy()

X = csr_matrix((np.ones(inds.shape[1]), (inds[0, :], inds[1, :])), shape = (N_USERS, N_ITEMS))
similarities = pairwise_jaccard_sparse(X, 0.87)
similarities_symm = (similarities + similarities.T).astype(bool).astype(int)

N, components = scipy.sparse.csgraph.connected_components(similarities_symm)

In [ ]:
# Create dictonary of users who have rated each item (i.e. invert train_dict)
train_dict = {}
test_dict = {}

with open(datapath("train.txt")) as file:
    for line in file:
        (user_id, items) = line.split(maxsplit=1)
        train_dict[user_id] = items.split()

with open(datapath("test.txt")) as file:
    for line in file:
        split = line.split(maxsplit=1)
        if (len(split) > 1):
            (user_id, items) = split
            test_dict[user_id] = items.split()
        else:
            test_dict[split[0]] = ''

train_dict = {int(k):[int(id) for id in v] for k,v in train_dict.items()}
test_dict = {int(k):[int(id) for id in v] for k,v in test_dict.items()}

item_users_dict = defaultdict(lambda: set())
for k,v in train_dict.items():
    for x in v:
        item_users_dict[x].add(k)

for k,v in test_dict.items():
    for x in v:
        item_users_dict[x].add(k)

In [ ]:
pd.Series([len(x) for x in item_users_dict.values()]).hist()
plt.yscale('log')

In [ ]:
comp_indices = np.vstack([np.arange(len(components)), components])

def get_local_neighbors(user_id):
    return set(np.arange(N_USERS)[(similarities_symm.getrow(user_id) != 0).toarray()[0]])

def get_global_neighbors(user_id):
    return set(np.where(components == components[user_id])[0])
    # comp_indices = np.vstack([np.arange(len(components)), components])
    # group = components[user_id]
    # return set(comp_indices[0, comp_indices[1, :] == group])

local_neighbors_map = {}
global_neighbors_map = {}
for user_id in tqdm(range(N_USERS)):
    local_neighbors_map[user_id] = get_local_neighbors(user_id)
    global_neighbors_map[user_id] = get_global_neighbors(user_id)

In [ ]:
def cliquify_components(csr):
    _, components = scipy.sparse.csgraph.connected_components(csr)

    values = []
    rs = []
    cs = []
    for i in tqdm(range(csr.shape[0])):
        neighbors = list(np.where(components == components[i])[0])
        neighbors.remove(i)
        cs += neighbors
        rs += [i] * len(neighbors)
        values += [1] * len(neighbors)

    return csr_matrix((values, (rs, cs)))

global_neighbors = cliquify_components(similarities_symm)

In [ ]:
# Unit test
rs = [0, 1, 1, 2, 3, 4, 4, 4, 5, 6]
cs = [1, 0, 2, 1, 4, 3, 5, 6, 4, 4]
vs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

cliquify_components(csr_matrix((vs, (rs, cs)))).A

In [ ]:
# for item in tqdm(range(N_ITEMS)):
#     rs = list(item_users_dict[0])
#     cs = [0] * len(rs)
#     vs = [1] * len(rs)
#     user_mat = csr_matrix((vs, (rs, cs)), shape = (N_USERS, 1))

#     L_uis = similarities_symm @ user_mat
#     G_uis = global_neighbors @ user_mat

#     break

In [ ]:
# This 100% can be made much much faster with vectorization 
# but I'm too lazy to think about that at this point

# Or maybe not, vectorized version still slow for some reason...

if (not Path(datapath("item_LGRs.pickle")).is_file()):
    item_LGRs = []
    item_UIS1s = []
    for item in tqdm(range(N_ITEMS)):
        LGRs = []
        UIS1s = []
        for user_id in range(N_USERS):
            local_neighbors = local_neighbors_map[user_id]
            global_neighbors = global_neighbors_map[user_id]
            L_ui = len(local_neighbors & item_users_dict[item])
            G_ui = len(global_neighbors & item_users_dict[item])
            N_i = len(item_users_dict[item])

            if (L_ui != 0 or G_ui != 0): LGRs.append(1 - L_ui / G_ui)
            if (L_ui != 0 or N_i != 0): UIS1s.append(1 - L_ui / N_i)

        item_LGRs.append(np.array(LGRs).mean())
        item_UIS1s.append(np.array(UIS1s).mean())

    with open(datapath("item_LGRs.pickle"), "wb") as f:
        pickle.dump(item_LGRs, f)

    with open(datapath("item_UIS1s.pickle"), "wb") as f:
        pickle.dump(item_UIS1s, f)

In [ ]:
with open(datapath("item_LGRs.pickle"), "rb") as f:
    item_LGRs = pickle.load(f)

with open(datapath("item_UIS1s.pickle"), "rb") as f:
    item_UIS1s = pickle.load(f)

#### LGR

In [ ]:
for drop in tqdm([5, 10, 15, 20, 25, 30]):
    q = np.quantile(item_LGRs, drop * 0.01)
    indices = np.where(item_LGRs < q)[0]

    dense_train_data = train_data.bool().to_dense().clone()
    dense_test_data = test_data.bool().to_dense().clone()
    dense_train_data[:, indices] = 0
    dense_test_data[:, indices] = 0
    torch.save(dense_train_data.to_sparse(), datapath(f"train/LGR/LGR{drop}_data.pt"))
    
    test_dict = sparse_to_user_dict(dense_test_data.to_sparse())

    with open(datapath(f"test/LGR/LGR{drop}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)

#### UIS1

In [ ]:
for drop in tqdm([5, 10, 15, 20, 25, 30]):
    q = np.quantile(item_UIS1s, drop * 0.01)
    indices = np.where(item_UIS1s < q)[0]

    dense_train_data = train_data.bool().to_dense().clone()
    dense_test_data = test_data.bool().to_dense().clone()
    dense_train_data[:, indices] = 0
    dense_test_data[:, indices] = 0
    torch.save(dense_train_data.to_sparse(), datapath(f"train/UIS1/UIS1{drop}_data.pt"))
    
    test_dict = sparse_to_user_dict(dense_test_data.to_sparse())

    with open(datapath(f"test/UIS1/UIS1{drop}_test_indices.pickle"), "wb") as f:
        pickle.dump(test_dict, f)

In [28]:
int(N_ITEMS * 0.2)

18319

### Subsample

In [32]:
np.random.seed(0)
items_to_keep = np.random.choice(range(N_ITEMS), size = int(N_ITEMS_SUB), replace = False)
item_map = {}
for i in range(len(items_to_keep)):
    item_map[items_to_keep[i]] = int(i)
def subsample_users_items(sparse_train_tensor, test_dict, num_users = N_USERS_SUB, items_to_keep = items_to_keep):
    indices = sparse_train_tensor.coalesce().indices().numpy()
    #print("here1")

    indices = indices[:, (indices[0] < num_users) & np.isin(indices[1], items_to_keep)]
    for i in range(indices.shape[1]):
        indices[1][i] = item_map[indices[1][i]]
    values = np.ones(indices.shape[1])

    #print("here2")

    test_dict_out = {
        k:
        list(np.array(v)[np.isin(np.array(v), items_to_keep)])
        for k,v in test_dict.items()
    }
    
    for k in list(test_dict_out.keys()):
        if k >= num_users:
            del test_dict_out[k]
        else:
            new_list = [item_map[x] for x in test_dict_out[k]]
            test_dict_out[k] = new_list

    #print(test_dict_out)
    #print("here3")

    return torch.sparse_coo_tensor(indices, values, size = (num_users, int(N_ITEMS_SUB))), test_dict_out



In [ ]:
# TODO: add whatever sparsity levels you want to end up using
to_modify = [
    "top/top70",
    "top/top80",
    "top/top90",
    "top/top95",
    "middle/middle5",
    "middle/middle10",
    "middle/middle15",
    "middle/middle25",
    "uniform/uniform100",
    "uniform/uniform99",
    "uniform/uniform95",
    "uniform/uniform90",
    "uniform/uniform80",
    "uniform/uniform70",
    "uniform/uniform60",
    "uniform/uniform50",
    "uniform/uniform40",
    "LGR/LGR5",
    "LGR/LGR10",
    "LGR/LGR15",
    "LGR/LGR25",
    "UIS1/UIS15",
    "UIS1/UIS110",
    "UIS1/UIS115",
    "UIS1/UIS125"
]

for file_name in to_modify:
    print(file_name)
    sparse_train_tensor = torch.load(datapath("train/" + file_name + "_data.pt"))
    with open(datapath("test/" + file_name + "_test_indices.pickle"), 'rb') as handle:
        test_dict = pickle.load(handle)

    new_train, new_test_dict = subsample_users_items(sparse_train_tensor, test_dict)

    # TODO: Modify file names as you want
    torch.save(new_train, datapath("train/" + file_name + "_data_sub.pt"))
    with open(datapath("test/" + file_name + "_test_indices_sub.pickle"), "wb") as f:
        pickle.dump(new_test_dict, f)

top/top70
top/top80
top/top90
top/top95
middle/middle5
middle/middle10
middle/middle15
middle/middle25
uniform/uniform100
uniform/uniform99
uniform/uniform95
uniform/uniform90
uniform/uniform80
uniform/uniform70
uniform/uniform60


In [ ]:
new_train

In [ ]:
list(map(lambda x: x+1 , [1,2,3]))